In [16]:
from ultralytics import YOLO
import cv2
import numpy as np

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can choose from 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', etc. 'yolov8n.pt' is the smallest and fastest.

# Initialize the webcam (0 is the default camera)
cap = cv2.VideoCapture(0)

# Check if the webcam is opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    # Capture each frame
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break
    
    # Run YOLOv8 model on the frame
    results = model(frame)

    output_frame = np.zeros_like(frame)
    
    # Parse and draw bounding boxes for human detections
    for result in results:
        for box in result.boxes:
            # YOLOv8 returns boxes with class names, confidence scores, and coordinates
            if box.cls == 0:  # YOLOv8 uses '0' for the "person" class in COCO
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f'Person: {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Copy the bounding box region to the output_frame
                output_frame[y1:y2, x1:x2] = frame[y1:y2, x1:x2]


                

    # Display the frame with bounding boxes
    cv2.imshow("YOLOv8 Human Detection", output_frame)
    
    # Break loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()



0: 480x640 3 persons, 186.2ms
Speed: 1.6ms preprocess, 186.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 146.0ms
Speed: 0.0ms preprocess, 146.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 151.8ms
Speed: 5.8ms preprocess, 151.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 chair, 154.4ms
Speed: 6.2ms preprocess, 154.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 chair, 1 tv, 161.5ms
Speed: 1.0ms preprocess, 161.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 chair, 1 tv, 167.2ms
Speed: 0.0ms preprocess, 167.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 chair, 158.4ms
Speed: 0.0ms preprocess, 158.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 tv, 179.1ms
Speed: 3.3ms

In [8]:


import cv2
import time

class MotionDetector:
    def __init__(self, threshold=150):
        self.threshold = threshold
        self.previous_frame = None
        self.start_time = None
        
    def detect_motion(self, frame):
        # Convert frame to grayscaleq
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Blur the grayscale image to reduce noise
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
        # If this is the first frame, initialize previous frame and return False
        if self.previous_frame is None:
            self.previous_frame = gray
            self.start_time = time.time()
            return False
        
        # Compute absolute difference between current frame and previous frame
        frame_diff = cv2.absdiff(self.previous_frame, gray)
        # Apply a threshold to the difference image to extract foreground regions
        thresholded = cv2.threshold(frame_diff, self.threshold, 255, cv2.THRESH_BINARY)[1]
        # Dilate the thresholded image to fill in holes
        thresholded = cv2.dilate(thresholded, None, iterations=2)
        
        # Find contours in the thresholded image
        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Check if any contours were found
        if len(contours) > 0:
            # Motion detected, update previous frame and return True
            self.previous_frame = gray
            self.start_time = time.time()
            return True
        
        # No motion detected, update previous frame and return False
        self.previous_frame = gray
        return False


# Initialize the MotionDetector object
md = MotionDetector()

# Open the laptop webcam
cap = cv2.VideoCapture(0)

start_time = cv2.getTickCount()
countdown_time = 15  # Countdown time in seconds
elapsed_time = 0
game_over = False
motiond = 0
end = 0

circle_color = (0, 255, 0)  # Green color
circle_radius = 50
circle_thickness = 5
circle_center = (100, 150)
last_color_change_time = time.time()

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can choose from 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', etc. 'yolov8n.pt' is the smallest and fastest.

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    print(ret)
    # If the frame was not captured, exit the loop
    if not ret:
        break
    
    # Resize the frame for faster processing
    frame = cv2.resize(frame, (1280, 720))

    
    # Run YOLOv8 model on the frame
    results = model(frame)

    output_frame = np.zeros_like(frame)
    
    # Parse and draw bounding boxes for human detections
    for result in results:
        for box in result.boxes:
            # YOLOv8 returns boxes with class names, confidence scores, and coordinates
            if box.cls == 0 and box.conf[0]>0.85:  # YOLOv8 uses '0' for the "person" class in COCO
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f'Person: {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Copy the bounding box region to the output_frame
                output_frame[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
    
    # frame = output_frame
    # Detect motion in the frame
    motion_detected = md.detect_motion(output_frame)

    # Check if the color of the circle should be changed
    current_time = time.time()
    if current_time - last_color_change_time > 3:
        last_color_change_time = current_time
        if circle_color == (0, 255, 0):
            circle_color = (0, 0, 255)  # Red color
        else:
            circle_color = (0, 255, 0)  # Green color
    
    # Draw the circle
    cv2.circle(frame, circle_center, circle_radius, circle_color, circle_thickness)
    
    # Display the countdown timer on the top left corner of the frame
    if not game_over:
        ##elapsed_time = time.time() - md.start_time
        elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        remaining_time = countdown_time - int(elapsed_time)
        if remaining_time <= 0 or motiond == 1:
            game_over = True
        else:
            cv2.putText(frame, "Time: {}".format(remaining_time), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            if cv2.waitKey(1) & 0xFF == ord('k'):
                
                cv2.putText(frame, "You Win!", (520, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

                win_start_time = time.time()
                while time.time() - win_start_time < 2:
                    cv2.imshow('Frame', frame)
                    cv2.waitKey(1)
                end = 1
                break

                
    
    # Display the text "detected" or "not detected" depending on whether motion is detected
    if motion_detected:
        if circle_color == (0, 0, 255):
           motiond = 1
        cv2.putText(frame, "Motion Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "Not Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Frame', frame)
    
    
    if game_over == True and end == 0:
        cv2.putText(frame, "Game Over", (480, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

        game_over_start_time = time.time()
        while time.time() - game_over_start_time < 2:
            cv2.imshow('Frame', frame)
            cv2.waitKey(1)
        end = 1
        break


    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()

True

0: 384x640 1 person, 200.1ms
Speed: 6.9ms preprocess, 200.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 148.7ms
Speed: 3.5ms preprocess, 148.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 150.1ms
Speed: 0.0ms preprocess, 150.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 160.1ms
Speed: 0.0ms preprocess, 160.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 174.6ms
Speed: 0.0ms preprocess, 174.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 156.8ms
Speed: 3.3ms preprocess, 156.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 178.4ms
Speed: 0.0ms preprocess, 178.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
True

0: 384x640 1 person, 152.7ms
Speed: 0.0ms preprocess, 152.7ms i

In [15]:

from ultralytics import YOLO
import numpy as np
import cv2
import time

class MotionDetector:
    def __init__(self, threshold=150):
        self.threshold = threshold
        self.previous_frame = None
        self.start_time = None
        
    def detect_motion(self, frame):
        # Convert frame to grayscaleq
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Blur the grayscale image to reduce noise
        gray = cv2.GaussianBlur(gray, (21, 21), 0)
        
        # If this is the first frame, initialize previous frame and return False
        if self.previous_frame is None:
            self.previous_frame = gray
            self.start_time = time.time()
            return False
        
        # Compute absolute difference between current frame and previous frame
        frame_diff = cv2.absdiff(self.previous_frame, gray)
        # Apply a threshold to the difference image to extract foreground regions
        thresholded = cv2.threshold(frame_diff, self.threshold, 255, cv2.THRESH_BINARY)[1]
        # Dilate the thresholded image to fill in holes
        thresholded = cv2.dilate(thresholded, None, iterations=2)
        
        # Find contours in the thresholded image
        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Check if any contours were found
        if len(contours) > 0:
            # Motion detected, update previous frame and return True
            self.previous_frame = gray
            self.start_time = time.time()
            return True
        
        # No motion detected, update previous frame and return False
        self.previous_frame = gray
        return False


# Initialize the MotionDetector object
md = MotionDetector()

# Open the laptop webcam
cap = cv2.VideoCapture(0)

start_time = cv2.getTickCount()
countdown_time = 60  # Countdown time in seconds
elapsed_time = 0
game_over = False
motiond = 0
end = 0

circle_color = (0, 255, 0)  # Green color
circle_radius = 50
circle_thickness = 5
circle_center = (100, 150)
last_color_change_time = time.time()

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # You can choose from 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', etc. 'yolov8n.pt' is the smallest and fastest.

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    # If the frame was not captured, exit the loop
    if not ret:
        break
    
    # Resize the frame for faster processing
    frame = cv2.resize(frame, (1280, 720))

    # Run YOLOv8 model on the frame
    results = model(frame)

    output_frame = np.zeros_like(frame)

    human_present = False
    
    # Parse and draw bounding boxes for human detections
    for result in results:
        for box in result.boxes:
            # YOLOv8 returns boxes with class names, confidence scores, and coordinates
            if box.cls == 0 and box.conf[0]>0.85:  # YOLOv8 uses '0' for the "person" class in COCO
                human_present = True
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0]
                
                # Draw the bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                label = f'Person: {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                # Copy the bounding box region to the output_frame
                output_frame[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
    
    if not human_present:
        cv2.putText(frame, "NO HUMAN PRESENT", (frame.shape[1]//2 - 250, frame.shape[0]//2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 12)
        cv2.putText(frame, "NO HUMAN PRESENT", (frame.shape[1]//2 - 250, frame.shape[0]//2), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 8)
        banda_aya = time.time()
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        last_color_change_time = time.time()
        continue

    # frame = output_frame
    # Detect motion in the frame
    motion_detected = md.detect_motion(output_frame)

    # Check if the color of the circle should be changed
    current_time = time.time()
    if current_time - last_color_change_time > 3:
        last_color_change_time = current_time
        if circle_color == (0, 255, 0):
            circle_color = (0, 0, 255)  # Red color
        else:
            circle_color = (0, 255, 0)  # Green color
    
    # Draw the circle
    cv2.circle(frame, circle_center, circle_radius, circle_color, circle_thickness)
    
    # Display the countdown timer on the top left corner of the frame
    if not game_over:
        ##elapsed_time = time.time() - md.start_time
        elapsed_time = (cv2.getTickCount() - start_time) / cv2.getTickFrequency()
        remaining_time = countdown_time - int(elapsed_time)
        if remaining_time <= 0 or motiond == 1:
            game_over = True
        else:
            cv2.putText(frame, "Time: {}".format(remaining_time), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            if cv2.waitKey(1) & 0xFF == ord('k'):
                
                cv2.putText(frame, "You Win!", (520, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

                win_start_time = time.time()
                while time.time() - win_start_time < 1:
                    cv2.imshow('Frame', frame)
                    cv2.waitKey(1)
                end = 1
                break

    # Display the text "detected" or "not detected" depending on whether motion is detected
    if motion_detected:
        if circle_color == (0, 0, 255):
           motiond = 1
        cv2.putText(frame, "Motion Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, "Not Detected", (1000, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the frame
    cv2.imshow('Frame', frame)

    if game_over == True and end == 0:
        cv2.putText(frame, "Game Over", (480, 360), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

        game_over_start_time = time.time()
        while time.time() - game_over_start_time < 1:
            cv2.imshow('Frame', frame)
            cv2.waitKey(1)
        end = 1
        break

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 216.6ms
Speed: 0.0ms preprocess, 216.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 182.5ms
Speed: 2.0ms preprocess, 182.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 1 tv, 157.0ms
Speed: 0.0ms preprocess, 157.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 1 tv, 182.2ms
Speed: 0.0ms preprocess, 182.2ms inference, 11.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 1 tv, 169.7ms
Speed: 0.0ms preprocess, 169.7ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 chair, 1 tv, 184.0ms
Speed: 0.0ms preprocess, 184.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tv, 175.8ms
Speed: 3.3ms preprocess, 175.8ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 chairs, 1 tv, 167.0ms
Speed: 1.3ms preprocess, 167.0ms inference, 0.0